# K-Means clutering technique to find PSD families within the CAMP2Ex field campain dataset

---

## Imports

In [2]:
import xarray as xr
import datatree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.transforms as mtransforms
from xhistogram.xarray import histogram
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score
from scipy.special import gamma
from dask.distributed import Client, LocalCluster
from matplotlib.colors import ListedColormap

In [6]:
# setting up the Seaborne style including figure  dpi
sns.set(rc={"figure.dpi":150, 'savefig.dpi':150})
sns.set(style='white', font_scale=0.9)
sns.set_style("ticks")

### Local Cluster

Let's spin up our `Dask` local cluster

In [7]:
cluster = LocalCluster()  
# display(cluster)

/data/keeling/a/alfonso8/miniconda3/envs/psd-retrievals/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 33083 instead
  warnings.warn(


## Data

CAMP2Ex dataset is store in Analysis-Ready Cloud-Optimized (ARCO) format ([Abernathey et al. 2021](https://ieeexplore.ieee.org/document/9354557)) using [Xarray-Datatree](https://xarray-datatree.readthedocs.io/en/latest/) data model that allows us to have both Learjet and P3B datasets in one `datatree`.

In [8]:
path_data = '../data/camp2ex_dtree2.zarr'
dt_camp2ex = datatree.open_datatree(path_data, engine='zarr', consolidated=True)

ValueError: Failed to decode variable 'time': unable to decode time units 'seconds since 2019-08-24T22:12:20' with "calendar 'proleptic_gregorian'". Try opening your dataset with decode_times=False or installing cftime if it is not installed.

In [5]:
dt_camp2ex.Lear

DataTree('Lear', parent="None")
    Dimensions:             (time: 4307, diameter: 58)
    Coordinates:
      * diameter            (diameter) float64 464B 50.0 60.0 ... 3.824e+03
      * time                (time) datetime64[ns] 34kB 2019-09-07T00:50:13 ... 20...
    Data variables: (12/17)
        Att_ka              (time) float64 34kB ...
        dbz_t_ka            (time) float64 34kB ...
        dbz_t_ku            (time) float64 34kB ...
        dm                  (time) float64 34kB ...
        dm_rt_dfr_gm_mu_3   (time) float64 34kB ...
        log10_nw            (time) float64 34kB ...
        ...                  ...
        r                   (time) float64 34kB ...
        r_dm_gm_mu_3        (time) float64 34kB ...
        r_gpm_operational   (time) float64 34kB ...
        sigma               (time) float64 34kB ...
        temp                (time) float64 34kB ...
        vert_vel            (time) float64 34kB ...

In [6]:
def sel_cols(ds):
    cols = ['sigma', 'dm', 'log10_nw', 'r', 'nt', 'lwc_cum', 'dbz_t_ku', 'dbz_t_ka', 'mu', 'Att_ka', 'temp', 'r_dm_gm_mu_3', 
            'r_gpm_operational', 'dm_rt_dfr_gm_mu_3', 'log10nw_dm_gm_mu_3', 'vert_vel', "lwc"]
    att = {"sigma" : {"units": "Unitless", "long_name": "Mass spectrum standard deviation", "description": "Mass spectrum standard deviation derived from in-situ particle size distribution measurements using Williams et al., (2014) methodology"}, 
       "dm" : {"units": "mm", "long_name": "Mass-weighted mean diameter", "description": "Mass-weighted mean diameter derived from in-situ particle size distribution measurements using Williams et al., (2014) methodology"}, 
       "log10_nw" : {"units": "log_10(mm-1 mm-3))", "long_name": "Normalized intercept parameter", "description": "Normalized intercept parameter derived from in-situ particle size distribution measurements using Williams et al., (2014) methodology"}, 
       "r" : {"units": "mmhr-1", "long_name": "rainfall rate", "description": "rainfall rate derived from in-situ particle size distribution measurements"}, 
       "nt" : {"units": "m-3", "long_name": "Total number concentration", "description": "Total number concentration derived from in-situ particle size distribution measurements"},
       "lwc" : {"units": "gm-3", "long_name": "liquid water content", "description": "liquid water content at each bin size derived from in-situ particle size distribution measurements"},
       "lwc_cum" : {"units": "gm-3", "long_name": "total liquid water content", "description": "total liquid water content derived from in-situ particle size distribution measurements"},
       "dbz_t_ku" : {"units": "dBZ", "long_name": "radar reflectivity at Ku-Band", "description": "radar reflectivity at Ku-Band derived from in-situ particle size distribution measurements"}, 
       "dbz_t_ka" : {"units": "dBZ", "long_name": "radar reflectivity at Ku-Band", "description": "radar reflectivity at Ka-Band derived from in-situ particle size distribution measurements"}, 
       "mu" : {"units": "unitless", "long_name": "shape parameter", "description": "shape parameter of the Normalized-Gamma size distribution derived from in-situ particle size distribution measurements"}, 
       "Att_ka" : {"units": "dB Km-1", "long_name": "Specific attenaution at Ka-band", "description": "Specific attenaution at Ka-band values derived from in-situ particle size distribution measurements"}, 
       "temp" : {"units": "°C", "long_name": "Ambient temperature", "description": "Ambient temperature in degrees from in-situ measurements"}, 
       "aircraft" : {"units": "None", "long_name": "Aircraft", "description": "Aircraft from which data was collected"},
       "r_dm_gm_mu_3" : {"units": "mmhr-1", "long_name": "rainfall rate", "description": "rainfall rates retrieved using GPM-Analytical retrieval (mu=3)"}, 
       "r_gpm_operational" : {"units": "mmhr-1", "long_name": "rainfall rate", "description": "rainfall rates retrieved using GPM-Operational retrieval (R-D relationships)"}, 
       "dm_rt_dfr_gm_mu_3" : {"units": "mm", "long_name": "Mass-weighted mean diameter", "description": "Mass-weighted mean diameter derived using GPM-Analytical retrieval (mu=3)"}, 
       "log10nw_dm_gm_mu_3" :{"units": "log_10(mm-1 mm-3))", "long_name": "Normalized intercept parameter", "description": "Normalized intercept parameter derived using GPM-Analytical retrieval (mu=3)"}, 
       "vert_vel" : {"units": "ms-1", "long_name": "vertical velocity", "description": "vertical velocity from in-situ measurements"}, 
       "time" : {"long_name": "Measurement time", "description": "Measurement time for in-situ measurements"},
       "diameter" : {"units": "mm", "long_name": "particle diameter", "description": "particle diameter at each bin size"},
       "d_d" : {"units": "mm", "long_name": "bin diameter width", "description": "bin diameter width"}
           
      }
    ds_new = ds[cols]
    for var in list(ds_new.variables):
        ds_new[var] = ds_new[var].assign_attrs(att[var])
    return ds_new

In [10]:
dt = dt_camp2ex.map_over_subtree(sel_cols)

In [11]:
dt.to_zarr('../data/camp2ex_dtree2.zarr')

In [ ]:
dt.Lear

In [7]:
cols = ['sigma', 'dm', 'log10_nw', 'r', 'nt', 'lwc_cum', 'dbz_t_ku', 'dbz_t_ka', 'mu', 'Att_ka', 'temp', 'aircraft', 'r_dm_gm_mu_3', 
        'r_gpm_operational', 'dm_rt_dfr_gm_mu_3', 'log10nw_dm_gm_mu_3', 'vert_vel']

In [10]:
# display(dt_camp2ex['Lear'].ds)
ds_lear = dt_camp2ex['Lear'].ds.copy()
ds_lear['aircraft'] = ('time', np.zeros_like(ds_lear.Att_ka.values))

In [11]:
# display(dt_camp2ex['P3B'].ds)
ds_p3 = dt_camp2ex['P3B'].ds.copy()
ds_p3['aircraft'] = ('time', np.ones_like(ds_p3.Att_ka.values))

Let's select the following fields we will use during the K-means clustering analysis and other variables we will use during our Deep Neural Network Training

Now we can merge both datasets into a single `Xarray.Dataset`

In [12]:
ds = xr.concat([ds_lear[cols], ds_p3[cols]], dim='time')

In [13]:
ds

<xarray.Dataset> Size: 1MB
Dimensions:             (time: 9453)
Coordinates:
  * time                (time) datetime64[ns] 76kB 2019-09-07T00:50:13 ... 20...
Data variables: (12/17)
    sigma               (time) float64 76kB 0.6386 0.6475 ... 0.5368 0.4025
    dm                  (time) float64 76kB 1.275 1.285 1.474 ... 1.482 0.9302
    log10_nw            (time) float64 76kB 3.836 3.962 3.788 ... 3.741 4.561
    r                   (time) float64 76kB 3.629 5.049 6.441 ... 6.102 4.338
    nt                  (time) float64 76kB 595.6 677.2 ... 544.8 2.781e+03
    lwc_cum             (time) float64 76kB 0.222 0.3071 ... 0.3261 0.3344
    ...                  ...
    aircraft            (time) float64 76kB 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    r_dm_gm_mu_3        (time) float64 76kB 2.167 3.298 nan ... 8.231 6.58 2.459
    r_gpm_operational   (time) float64 76kB 10.85 11.06 nan ... 5.679 2.412
    dm_rt_dfr_gm_mu_3   (time) float64 76kB 1.465 1.47 nan ... 1.305 1.3 1.11
    log10nw_dm_gm_mu_3  (time) float64 76kB 33.51 35.27 nan ... 40.61 39.55
    vert_vel            (time) float64 76kB -0.2 -0.4 -0.7 ... -5.4 -5.6 -5.9

We discarded data with Liquid Water Content  $LWC <=0.05 gm^{-3}$ (Lance et at., 2010, Gupta et al 2021) and take $log_{10}$ of rainfall rate (r), total number concentration (nt) and liquid water content (lwc_cum)

In [14]:
ds = ds.where(ds.lwc_cum > 0.05, drop=True)
ds = ds
ds['logr'] = np.log10(ds.r)
ds['lognt'] = np.log10(ds.nt)
ds['loglwc'] = np.log10(ds.lwc_cum)


Now we converted our `Xarray.Dataset` into a `Panda.Dataframe`

In [16]:
df = ds.to_dataframe().reset_index()
df

,time,sigma,dm,log10_nw,r,nt,lwc_cum,dbz_t_ku,dbz_t_ka,mu,...,temp,aircraft,r_dm_gm_mu_3,r_gpm_operational,dm_rt_dfr_gm_mu_3,log10nw_dm_gm_mu_3,vert_vel,logr,lognt,loglwc
0,2019-09-07 00:50:13,0.638638,1.274745,3.835770,3.628710,595.554870,0.222011,31.954252,32.279607,-0.015834,...,20.20,0.0,2.167368,10.853267,1.465,33.506095,-0.2,0.559752,2.774922,-0.653626
1,2019-09-07 00:50:14,0.647497,1.285212,3.962426,5.048991,677.244280,0.307068,33.836180,33.265364,-0.060199,...,20.20,0.0,3.298437,11.055554,1.470,35.266054,-0.4,0.703205,2.830745,-0.512765
2,2019-09-07 00:50:15,0.719944,1.473846,3.787725,6.441170,602.352457,0.355175,36.060344,35.180436,0.190898,...,20.00,0.0,NaN,NaN,NaN,NaN,-0.7,0.808965,2.779851,-0.449557
3,2019-09-07 00:50:16,0.755878,1.562636,3.735953,7.516483,611.128850,0.398377,37.370368,36.037833,0.273775,...,19.60,0.0,NaN,NaN,NaN,NaN,-0.6,0.876015,2.786133,-0.399706
4,2019-09-07 00:50:17,0.268457,1.070191,4.282757,4.585315,627.189980,0.308683,28.851578,30.202350,11.891776,...,19.20,0.0,2.565654,5.679283,1.300,36.465796,-0.6,0.661369,2.797399,-0.510487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7897,2019-10-05 07:14:23,0.385260,1.222033,3.872777,3.343195,399.900000,0.204183,29.070989,30.839838,6.061416,...,2.59,1.0,4.335463,1.485137,1.015,43.727699,-5.1,0.524162,2.601951,-0.689980
7898,2019-10-05 07:14:26,0.334772,1.581900,3.643099,6.776052,304.700000,0.337851,33.969278,35.924312,18.328463,...,2.83,1.0,NaN,NaN,NaN,NaN,-4.9,0.830977,2.483872,-0.471275
7899,2019-10-05 07:14:27,0.361556,1.489003,3.804213,7.357044,429.800000,0.384331,33.899526,35.797127,12.960578,...,2.99,1.0,8.231469,5.798685,1.305,41.450285,-5.4,0.866703,2.633266,-0.415295
7900,2019-10-05 07:14:28,0.536839,1.482319,3.740727,6.102195,544.800000,0.326140,33.948309,35.663458,3.624216,...,3.11,1.0,6.580222,5.679283,1.300,40.606289,-5.6,0.785486,2.736237,-0.486596


In [ ]:
fig, (ax, ax1, ax3) = plt.subplots(1, 3, figsize=(12, 4))
sc = ax.scatter(df.dbz_t_ku, df.Att_ka, c=df.dm, s=2, cmap='jet')
ax.set_yscale('log')
fig.colorbar(sc, ax=ax)

sc1 = ax1.scatter(df.dbz_t_ku, df.Att_ka, c=df.lwc_cum, s=2, vmin=0, vmax=4, cmap='jet')
ax1.set_yscale('log')
fig.colorbar(sc1, ax=ax1)


sc3 = ax3.scatter(df.dbz_t_ku, df.Att_ka, c=df.log10_nw, s=2,  cmap='jet')
ax3.set_yscale('log')
fig.colorbar(sc3, ax=ax3)


In [ ]:
df['mu_3']= 3

br = np.arange(0.5, 5.5, 0.001)
res = np.zeros_like(br)
for i in range(br.shape[0]):
        res[i] = np.corrcoef(df.dm, df.sigma / df.dm ** br[i])[0, 1] ** 2
res

In [ ]:
bm = br[np.argmin(res)]
bm

In [ ]:
df['sigma_prime'] = df.sigma / (df.dm ** bm)
df['new_sigma'] = df['sigma_prime'].mean() * df['dm'] ** bm
df['mu_unc'] = (df.dm ** (2 - 2 * bm) / (df['sigma_prime'].mean() ** 2)) - 4

## K-means

To apply the cluster analysis, we standardized our input features by removing the mean and scaling to unit variance using the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from the `Sklearn` Python package

In [ ]:
scaler = StandardScaler()
df[['sigma_T', 'dm_T', 'log10_nw_T', 'logr_T', 'lognt_T', "loglwc_T"]]= scaler.fit_transform(df[['sigma', 'dm', 'log10_nw', 'logr', 'lognt', 'loglwc']])

### K-means clustering benchmarking

As a supervised machine learning technique, K-means clustering requires the number of the cluster to be defined beforehand. To determine the optimal number of clusters (k) for the PSDs, we executed the algorithm for k values ranging from 2 to 15. Using the within-cluster sum of squares (WCSS), also known as the elbow method, Davies-Bouldin index (Davies & Bouldin, 1979), and Silhouette score (Rousseeuw, 1987)

In [ ]:
def get_kmeans_score(df, center):
    '''
    returns the elbow inertial index, the Davies Bouldin and Silhouette score
    INPUT:
        data - the dataset you want to fit kmeans to
        center - the number of centers you want (the k value)
    OUTPUT:
        elbow inertial index, the Davies Bouldin and Silhouette score
    '''
    kmeans = KMeans(n_clusters=center, random_state=10)
    model = kmeans.fit(df)
    model2 = kmeans.fit_predict(df)
    cluster_labels = model.labels_
    
    dav = davies_bouldin_score(df, model2)
    sil = silhouette_score(df, cluster_labels)
    elbow = model.inertia_
    return dav, sil, elbow

We defined some list to store results of each cluster results for every score. Then we test each number of cluster. 

In [ ]:
dav = []
sil = []
elbow = []

for k in range(2,15):
    _dav, _sil, _el = get_kmeans_score(df[['sigma_T', 'dm_T', 'log10_nw_T', 'logr_T', 'lognt_T', "loglwc_T"]], k)
    dav.append(_dav)
    sil.append(_sil)
    elbow.append(_el)

Now, we can see the score result for different number of clusters.


In [ ]:
centers = range(2,15)
fig, (ax, ax1, ax2) = plt.subplots(1, 3, figsize=(12, 4), dpi=100)
ax.plot(centers, dav, linestyle='--', marker='o', color='b');
ax.set_xlabel('K');
ax.set_ylabel('Score');
ax.set_title('Davies Bouldin method');

ax1.plot(centers, sil, linestyle='--', marker='o', color='b');
ax1.set_xlabel('K');
ax1.set_ylabel('Score');
ax1.set_title('silhouette method');

ax2.plot(centers, elbow, linestyle='--', marker='o', color='b');
ax2.set_xlabel('K');
ax2.set_ylabel('Score');
ax2.set_title('Elbow method');
fig.tight_layout()

### K-means clustering with 6 PSD families


Based on the cluster benchmarking, we deduced that k=6 the most suitable number of clusters representing the data.

In [ ]:
# select scaled/transformed data
X = df[['sigma_T', 'dm_T', 'log10_nw_T', 'logr_T', 'lognt_T', "loglwc_T"]]


We can now apply the K-means cluster technique using these (X) features

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=10)
kmeans.fit(X)

Create a new column with the Kmeans labels. We add one to have labales from 1 to 6 (instead of 0 to 5)

In [ ]:
df['kmeans_6'] = kmeans.labels_ + 1

In [ ]:
# Reorder and replace some labels to make them equal when plotting mean PSDs
df['kmeans'] = df['kmeans_6'].replace([1, 3, 4, 5, 2, 6], 
                                      [1, 2, 3, 4, 5, 6])

# computing Dual Frequency Ratio
df['dfr'] = df['dbz_t_ku'] - df['dbz_t_ka']

In [ ]:
df.shape

In [ ]:
# function that computes the Normalized-Gama size distribution
def norm_gamma(d, nw, mu, dm):
    """
    Functions that computes the normalized-gamma size distritubion (Testud et al., 2002)
    Param d: diameter in mm
    Param nw: Normalized intercep parameter
    Param mu: Shape parameter
    Param dm: Mass-weighted mean diameter
    """
    f_mu = (6 * (4 + mu) ** (mu + 4)) / (4 ** 4 * gamma(mu + 4) )
    slope = (4 + mu) / dm
    return nw * f_mu * (d / dm) ** mu * np.exp(-slope * d)

### K-means results 

Scatter plot of Dm and Nw colored by each PSD family is plotted as following. Mean PSD computed using the mean quantities of each parameter at each group is also displayed

In [ ]:
# number of clusters
n_c = 6
# defining the Colormap for each cluster identified
my_cmap6 = ListedColormap(sns.color_palette('deep', n_c))
colors6 = my_cmap6(np.linspace(0,1, n_c))

In [ ]:
# Plotting results 
fig, axs = plt.subplot_mosaic([['a)', 'b)']], figsize=(8,4))

# left panel
ax = axs['a)']
# Scatter plot of Dm and Nw
ax = sns.scatterplot(data=df, x=df['dm'], y=df['log10_nw'], hue=df['kmeans'], s=3,
                     ax=ax,palette=sns.color_palette('deep', 6), 
                     legend=False, edgecolor=None)
# plt.setp(ax.collections, alpha=0.4)
ax.set_xlabel(r"$D_m \ [mm]$")
ax.set_ylabel(r"$Log_{10}(Nw) \ [Log_{10}(mm^{1}mm^{-3})]$")
ax.grid('both', linestyle='--', lw=0.5, dashes=[7,7])

dms = np.linspace(df['dm'].min(), df['dm'].max(), 100)

# Plotting Bringi et al (2009) convective-stratiform separation
s_c = -1.6 * dms + 6.3
ax.plot(dms, s_c, c='k', ls='-.', lw=0.8, label=r"$Bringi \ et \ al. \ (2009)$")
ax.legend()

# right panel
ax1 = axs['b)']
ax1.set_yscale('log')
ax1.set_ylim(1e-3, 1e9)
d = dt_camp2ex['Lear'].ds.diameter/1000
ax1.grid('both')
ax1.set_ylabel(r"$N(D) \  [mm^{-1}m^{-3}]$")
ax1.set_xlabel(r"$D \ [mm]$")
ax1.grid('both', linestyle='--', lw=0.5, dashes=[7,7])
ax1.set_xlim(-0.2, 3)

# computing the mean particle size distribution for each group
for i in range(1, n_c + 1):
    df_sub = df[df['kmeans'] == i]
    mu = df_sub['mu_unc'].quantile(0.5)
    # mu = df_sub['mu'].quantile(0.5)

    dm = df_sub['dm'].quantile(0.5)
    nw = (10 ** (df_sub['log10_nw'])).quantile(0.5)
    gm = norm_gamma(d, nw=nw, mu=mu, dm=dm)
    ax1.plot(d, gm, c=colors6[i-1], label=f"Group {i}")


lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]


fig.legend(lines[1:], labels[1:], loc='upper center', ncol=6, bbox_to_anchor=[0.5, 1.05])
for label, ax in axs.items():
    # label physical distance in and down:
    trans = mtransforms.ScaledTranslation(-45/72, -1/72, fig.dpi_scale_trans)
    ax.text(0.0, 1.05, label, transform=ax.transAxes + trans,
            fontsize='small', verticalalignment='top')
fig.tight_layout()
# plt.savefig('../images/kmeans_scatter.svg',  bbox_inches='tight')

## Dataset Imbalance

It is safe to check data imbalance before performing a machine learning algorithm. We set a $1 mm$ treshold in $D_m$ for counting the number of PSDs within each category. 

In [ ]:
# creating a categorical variable to split data into greater and smaller Dm
df['dm_class'] = (df.dm >= 1.0).astype(int)

Then, we can create a two-dimension histogram to see the density distribution of our dataset. Also, we can include a bar diagram with the two classess we previously defined ($D_m >= 1mm$ and $D_m < 1mm$)

In [ ]:
# Creating the 2D-histogram inputs
xbins = np.linspace(ds.dm.min(), ds.dm.max(), 50)
ybins = np.linspace(ds.log10_nw.min(), ds.log10_nw.max(), 50)
psd = histogram(ds.dm, ds.log10_nw, bins=[xbins, ybins])

In [ ]:
# Plotting data imbalance results
fig, ax = plt.subplots(figsize=(5.5,4.5))

# 2D-histogram
im = psd.T.where(psd.T > 0, np.nan).plot(add_colorbar=False, ax=ax, cmap='magma_r', vmin=0, vmax=100)
fig.colorbar(im , ax=ax, label=r"$Counts$")
ax.set_xlabel(r"$D_m \ [mm]$")
ax.set_ylabel(r"$Log_{10}(Nw) \ [Log_{10}(mm^{1}mm^{-3})]$")
sns.despine()
ax.grid('both', linestyle='--', lw=0.5, dashes=[7,7])
ax.set_xlim(-0.1, 2.8)
ax.set_ylim(2, 11)
ax.vlines(x=1, ymin=2, ymax=11, lw=0.5, linestyle='--', color='k')

# Bar plot
l, b, h, w = .45, .60, .15, .3
ax2 = fig.add_axes([l, b, w, h])
bar_colors = ['tab:red', 'tab:blue']
ax2.bar([r'$D_m < 1.0$', r"$D_m \geq 1.0$"], np.bincount(df['dm_class']), color=bar_colors)
ax2.set_title("Counts")

## Median values for the key parameters

Including $D_m$, $N_w$, $μ_{unc}$, $σ$,$R$, $N_t$ and $v$ for each PSD family identified using the clustering technique.

In [ ]:
df[['kmeans', 'dm','log10_nw', 'mu_unc', 'sigma', 'r', 'lognt', 'vert_vel']].groupby('kmeans').median()

### Saving dataframe

We saved the Kmeans output and dataset imbalance results for further analysis

In [ ]:
df.to_parquet('../data/df_cluster.parquet')